# Prediction request terhadap Model yang Terdeploy

## Library Import

In [6]:
import requests
import tensorflow as tf
import base64
import numpy as np

## Definisikan URL Prediksi dan Mapping Label

In [ ]:
# URL model di Railway
MODEL_URL = "https://drugclassification-production.up.railway.app/v1/models/drug_classification:predict"

# Label sesuai urutan output model (5 kelas)
LABELS = ["DrugY", "drugA", "drugB", "drugC", "drugX"]

## Definisikan Sampel Data

In [8]:
# Contoh data
sample = {
    "Age": 23,
    "Sex": "F",
    "BP": "HIGH",
    "Cholesterol": "HIGH",
    "Na_to_K": 25.355
}

## Definisikan Fungsi Pemrosesan Data dan Request Prediksi

In [9]:
def make_tf_example(data):
    """
    Convert dictionary into tf.train.Example
    """
    feature = {
        "Age": tf.train.Feature(int64_list=tf.train.Int64List(value=[data["Age"]])),
        "Sex": tf.train.Feature(bytes_list=tf.train.BytesList(value=[data["Sex"].encode()])),
        "BP": tf.train.Feature(bytes_list=tf.train.BytesList(value=[data["BP"].encode()])),
        "Cholesterol": tf.train.Feature(bytes_list=tf.train.BytesList(value=[data["Cholesterol"].encode()])),
        "Na_to_K": tf.train.Feature(float_list=tf.train.FloatList(value=[data["Na_to_K"]]))
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def predict(sample):
    # Buat tf.train.Example serialized string
    serialized = make_tf_example(sample)

    # Encode ke base64 agar bisa diterima Serving
    b64_example = base64.b64encode(serialized).decode("utf-8")

    payload = {"instances": [ {"b64": b64_example} ]}

    response = requests.post(MODEL_URL, json=payload)
    if response.status_code == 200:
        result = response.json()["predictions"][0]
        probs = np.array(result)
        predicted_index = int(np.argmax(probs))
        predicted_label = LABELS[predicted_index]
        
        print("Probabilities:", dict(zip(LABELS, probs)))
        print("Predicted Label:", predicted_label)
    else:
        print("Error:", response.text)

## Lakukan Prediction Request

In [10]:
if __name__ == "__main__":
    predict(sample)

Probabilities: {'DrugY': 0.999999881, 'drugA': 3.87540524e-08, 'drugB': 1.68109118e-08, 'drugC': 5.40844081e-09, 'drugX': 7.77183544e-08}
Predicted Label: DrugY
